<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://www.nilu.no/">     
           <img 
               src="http://www.nilu.no/Portals/0/Logo_transparent.png" 
               style="width:100%"
           >
           </a>    
       </td>

   </tr>
</table>
<div style="float:right;"><h2>Retriving data from EBAS:</h2><h3>03. Carbon Dioxide and Methane</h3></div>

**We will now use data from Zeppelin (Ny-Aalesund) - 2016**

The observations made are part of the national monitoring programme conducted by NILU on behalf of The Norwegian Environment Agency.

The monitoring programme includes measurements of 41 greenhouse gases at the Zeppelin
Observatory in the Arctic; and this includes a long list of halocarbons, which are not only
greenhouse gases but also ozone depleting substances.

In [ ]:
# As always, we import the modules we need
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from netCDF4 import num2date
from pydap.client import open_dods, open_url

In [ ]:
output_notebook()

**We start with the carbon dioxide data**

In [ ]:
ds = open_dods('http://dev-ebas-pydap.nilu.no/NO0042G.online_crds.IMG.air.carbon_dioxide.1h.NO01L_picarro2_G2401.NO01L_picarro.2.dods')

keys = ds.keys()

for key in keys:
    print(key)

**Let's get the carbon_dioxide_amean and create some varibles for plotting**

In [ ]:
carbon_dioxide_amean = ds['carbon_dioxide_amean']

y_carbon_dioxide = carbon_dioxide_amean.carbon_dioxide_amean.data

x_carbon_dioxide = num2date(carbon_dioxide_amean.time.data,units='days since 1900-01-01 00:00:00',calendar='gregorian')

**Let's have a look at the metadata in order to determine the unit**

In [ ]:
ds['carbon_dioxide_amean_ebasmetadata'].keys()

In [ ]:
carbon_dioxide_metadata = ds['carbon_dioxide_amean_ebasmetadata'].carbon_dioxide_amean_ebasmetadata.data
print(len(carbon_dioxide_metadata))
#No wavelenghts this time, so there is just one element in the list
print(carbon_dioxide_metadata[0])

**Now that we have the variables and unit, we can plot the data**

In [ ]:
plot = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title='carbon_dioxide_zeppelin')

plot.line(x_carbon_dioxide, y_carbon_dioxide, line_dash="4 4", line_width=1, color='green')

plot.xaxis.axis_label = 'Date'
plot.yaxis.axis_label = 'umol/mol'

show(plot)

**We can also do a more advanced plot**

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool

plot = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title='Carbon Dioxide Arithmetic Mean')

plot.line(x_carbon_dioxide, y_carbon_dioxide, line_dash="4 4", line_width=1, color='gray')

plot.xaxis.axis_label = 'Date'
plot.yaxis.axis_label = 'umol/mol'

dots = plot.circle(x_carbon_dioxide, y_carbon_dioxide, size=10,
                fill_color="grey", hover_fill_color="firebrick",
                fill_alpha=0.05, hover_alpha=0.3,
                line_color=None, hover_line_color="white")

# We add a hover tool for the dots, so we matches everything that matches in a horizontal line we can alos do this vertically 
#plot.add_tools(HoverTool(tooltips=None, renderers=[dots], mode='vline'))

plot.add_tools(HoverTool(tooltips=None, renderers=[dots], mode='hline'))

show(plot)

**We can now have a look at the methane data for 2016**

In [ ]:
ds = open_dods('http://dev-ebas-pydap.nilu.no/NO0042G.online_crds.IMG.air.methane.1h.NO01L_picarro2_G2401.NO01L_picarro.2.dods')

for key in ds.keys():
    print(key)

**Again we append the data to variables in order to plot the data**
Note that we have a methane_amean_qc variable, this contains the flags. E.g. the data could be flagged missing data, Construction/acitivity nearby etc. Here is a full list of flags used in EBAS: http://ebas-submit.nilu.no/Submit-Data/List-of-Data-flags

**Let's have a quick look at the flags:**

In [ ]:
methane_qc_2016 = ds['methane_amean_qc']
print(methane_qc_2016)

In [ ]:
# Flag 653: Sampling period shorter than normal, considered representative. Observed values reported
methane_qc_2016.methane_amean_qc.data

**Okay, lets create methane variables for plotting:**

In [ ]:
methane_amean_2016 = ds['methane_amean']
methane_amean_2016.keys()

In [ ]:
x_methane = num2date(methane_amean_2016.time.data,units='days since 1900-01-01 00:00:00',calendar='gregorian')
y_methane = methane_amean_2016.methane_amean.data

**Let's have a look at the metadata in order to find the unit for the y-axis**

In [ ]:
methane_metadata = ds['methane_amean_ebasmetadata']

print(methane_metadata.keys())

In [ ]:
print(len(methane_metadata.methane_amean_ebasmetadata.data))
print(methane_metadata.methane_amean_ebasmetadata.data[0])

**Now we can create a simple plot**

In [ ]:
plot = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title='Methane 2016 Zeppelin')

plot.line(x_methane, y_methane, line_dash="4 4", line_width=1, color='black')

plot.xaxis.axis_label = 'Date'
plot.yaxis.axis_label = 'nmol/mol'

show(plot)

**Let's compare two specific timeperiods for both the carbon dioxide and methane time series**

In [ ]:
from bokeh.io import gridplot
from bokeh.models import ColumnDataSource

x = x_carbon_dioxide
y0 = y_carbon_dioxide
y1 = y_methane

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS='pan,wheel_zoom,xbox_select,reset'

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=450, plot_height=400, title="Carbon dioxide Zeppelin", x_axis_type="datetime")
left.circle('x', 'y0', source=source, selection_color="orange")

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=450, plot_height=400, title="Methane Zeppelin", x_axis_type="datetime")
right.circle('x', 'y1', source=source, selection_color="orange")

left.xaxis.axis_label = 'Date'
left.yaxis.axis_label = 'umol/mol'

right.xaxis.axis_label = 'Date'
right.yaxis.axis_label = 'nmol/mol'

p = gridplot([[left, right]])

show(p)